In [1]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler , MinMaxScaler
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder





from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_curve, auc, plot_confusion_matrix
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

In [2]:
df=pd.read_csv('/content/drive/MyDrive/colab_kagel/dataa/train_new.csv')
#df is the traonong data
df2=pd.read_csv('/content/drive/MyDrive/colab_kagel/dataa/test_new.csv')
# df2 is the test data
#to get the id for later use
sample=pd.read_csv('/content/drive/MyDrive/colab_kagel/dataa/sample_submission_new.csv')


In [3]:
df.head()

,price,retail_price,currency_buyer,units_sold,uses_ad_boosts,rating,rating_count,badges_count,badge_local_product,badge_product_quality,badge_fast_shipping,tags,product_color,product_variation_size_id,product_variation_inventory,shipping_option_name,shipping_option_price,shipping_is_express,countries_shipped_to,inventory_total,has_urgency_banner,urgency_text,origin_country,merchant_title,merchant_name,merchant_info_subtitle,merchant_rating_count,merchant_rating,merchant_id,merchant_has_profile_picture,merchant_profile_picture,theme,crawl_month,id
0,8.00,7,EUR,10000,1,4.0,1670,0,0,0,0,"Summer,soildcolor,Plus Size,Tank,camisole,Tops...",yellow,M,50,Livraison standard,3,0,30,50,NaN,NaN,CN,keepahorse,keepahorse,"88 % avis positifs (66,644 notes)",66644,4.137582,577fb2b368116418674befd9,1,https://s3-us-west-1.amazonaws.com/sweeper-pro...,summer,2020-08,272
1,8.00,7,EUR,1000,0,4.0,843,1,1,0,0,"bathing suit,Plus Size,bikini set,sexy swimsui...",black,L,50,Livraison standard,3,0,37,50,NaN,NaN,CN,shanghaimingqidianqikejiyouxiangongsi,上海铭启电气科技有限公司,"91 % avis positifs (25,752 notes)",25752,4.256873,566a3ef17233ff2686443082,0,NaN,summer,2020-08,1515
2,6.00,34,EUR,100,0,4.0,10,0,0,0,0,"Summer,Vest,momshirt,Get,summer t-shirts,funny...",white,L,50,Livraison standard,2,0,41,50,NaN,NaN,CN,zhaodongmei,zhaodongmei,"83 % avis positifs (3,897 notes)",3897,3.928920,5aacd6c1ccf0c83e5a6f3de0,0,NaN,summer,2020-08,940
3,4.67,4,EUR,100,1,3.0,82,0,0,0,0,"Summer,Shorts,pants,Beach,Plus Size,beachpant,...",lakeblue,XS,5,Livraison standard,1,0,25,50,NaN,NaN,CN,pookie0331,pookie0331,"87 % avis positifs (7,497 notes)",7497,4.079365,583141fbfef4094e51453d9b,0,NaN,summer,2020-08,309
4,8.00,11,EUR,1000,0,4.0,127,0,0,0,0,"Summer,Floral print,women dresses,fashion dres...",apricot,S,50,Livraison standard,3,0,15,50,1.0,Quantité limitée !,CN,shitongyi12683626,shitongyi12683626,91 % avis positifs (173 notes),173,4.306358,5d3bc9f8e13a7e654424a4cb,0,NaN,summer,2020-08,226


In [4]:
df.shape

(1094, 34)

In [5]:
df.describe()

,price,retail_price,units_sold,uses_ad_boosts,rating,rating_count,badges_count,badge_local_product,badge_product_quality,badge_fast_shipping,product_variation_inventory,shipping_option_price,shipping_is_express,countries_shipped_to,inventory_total,has_urgency_banner,merchant_rating_count,merchant_rating,merchant_has_profile_picture,id
count,1094.000000,1094.000000,1094.000000,1094.000000,1094.000000,1094.000000,1094.000000,1094.000000,1094.000000,1094.000000,1094.000000,1094.000000,1094.000000,1094.000000,1094.000000,349.0,1.094000e+03,1094.000000,1094.000000,1094.000000
mean,8.320786,23.712066,4518.661792,0.432358,4.006399,915.523766,0.114260,0.019196,0.083181,0.011883,32.807130,2.344607,0.002742,40.484461,49.849177,1.0,2.678399e+04,4.036191,0.155393,772.207495
std,4.027448,31.802661,9825.608568,0.495630,0.585706,2037.561158,0.348461,0.137275,0.276282,0.108409,21.454655,1.044781,0.052318,20.196229,2.480687,0.0,8.616171e+04,0.200397,0.362445,453.784430
min,1.000000,2.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,8.000000,1.000000,1.0,4.000000e+00,2.941176,0.000000,2.000000
25%,5.830000,7.000000,100.000000,0.000000,4.000000,24.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2.000000,0.000000,31.000000,50.000000,1.0,1.926000e+03,3.918464,0.000000,382.250000
50%,8.000000,10.000000,1000.000000,0.000000,4.000000,143.500000,0.000000,0.000000,0.000000,0.000000,50.000000,2.000000,0.000000,40.000000,50.000000,1.0,8.225000e+03,4.045170,0.000000,752.500000
75%,11.000000,26.000000,5000.000000,1.000000,4.000000,845.500000,0.000000,0.000000,0.000000,0.000000,50.000000,3.000000,0.000000,43.000000,50.000000,1.0,2.306200e+04,4.166826,0.000000,1166.500000
max,49.000000,252.000000,100000.000000,1.000000,6.000000,20744.000000,3.000000,1.000000,1.000000,1.000000,50.000000,12.000000,1.000000,140.000000,50.000000,1.0,2.174765e+06,4.577519,1.000000,1572.000000


In [6]:
# The full summary
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1094 entries, 0 to 1093
Data columns (total 34 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   price                         1094 non-null   float64
 1   retail_price                  1094 non-null   int64  
 2   currency_buyer                1094 non-null   object 
 3   units_sold                    1094 non-null   int64  
 4   uses_ad_boosts                1094 non-null   int64  
 5   rating                        1094 non-null   float64
 6   rating_count                  1094 non-null   int64  
 7   badges_count                  1094 non-null   int64  
 8   badge_local_product           1094 non-null   int64  
 9   badge_product_quality         1094 non-null   int64  
 10  badge_fast_shipping           1094 non-null   int64  
 11  tags                          1094 non-null   object 
 12  product_color                 1065 non-null   object 
 13  pro

In [7]:
df.corr().loc['rating']

price                           0.226129
retail_price                    0.086251
units_sold                      0.011729
uses_ad_boosts                 -0.047359
rating                          1.000000
rating_count                    0.025344
badges_count                    0.234001
badge_local_product             0.009850
badge_product_quality           0.273749
badge_fast_shipping             0.042029
product_variation_inventory     0.048443
shipping_option_price           0.202719
shipping_is_express             0.059141
countries_shipped_to            0.024565
inventory_total                -0.116458
has_urgency_banner                   NaN
merchant_rating_count           0.052712
merchant_rating                 0.216270
merchant_has_profile_picture    0.025481
id                              0.058845
Name: rating, dtype: float64

In [8]:
df.isnull().sum()

price                             0
retail_price                      0
currency_buyer                    0
units_sold                        0
uses_ad_boosts                    0
rating                            0
rating_count                      0
badges_count                      0
badge_local_product               0
badge_product_quality             0
badge_fast_shipping               0
tags                              0
product_color                    29
product_variation_size_id        12
product_variation_inventory       0
shipping_option_name              0
shipping_option_price             0
shipping_is_express               0
countries_shipped_to              0
inventory_total                   0
has_urgency_banner              745
urgency_text                    745
origin_country                   12
merchant_title                    0
merchant_name                     4
merchant_info_subtitle            0
merchant_rating_count             0
merchant_rating             

In [9]:
df.nunique()

price                            117
retail_price                      95
currency_buyer                     1
units_sold                        13
uses_ad_boosts                     2
rating                             6
rating_count                     584
badges_count                       4
badge_local_product                2
badge_product_quality              2
badge_fast_shipping                2
tags                             895
product_color                     87
product_variation_size_id         83
product_variation_inventory       47
shipping_option_name              13
shipping_option_price              8
shipping_is_express                2
countries_shipped_to              83
inventory_total                    6
has_urgency_banner                 1
urgency_text                       2
origin_country                     5
merchant_title                   747
merchant_name                    746
merchant_info_subtitle           808
merchant_rating_count            718
m

In [10]:
# df= df.drop(df.loc[:,df.nunique()==1].keys(), axis=1)

In [11]:
df= df.drop(['currency_buyer','has_urgency_banner','theme','crawl_month','merchant_profile_picture','merchant_info_subtitle','tags','merchant_name','urgency_text','id'], axis=1)

In [12]:
df.shape

(1094, 24)

In [13]:
df=df.fillna(method='bfill', axis=0)

In [14]:
df.isnull().values.any()

False

In [15]:
df['product_color']= LabelEncoder().fit_transform(df['product_color']) 
df['product_color']

0       86
1       12
2       77
3       40
4        8
        ..
1089    68
1090    66
1091    66
1092    12
1093    86
Name: product_color, Length: 1094, dtype: int64

In [16]:
df['merchant_title']= LabelEncoder().fit_transform(df['merchant_title']) 
df['merchant_title']

0       472
1       584
2       725
3       556
4       597
       ... 
1089    399
1090    746
1091    187
1092    708
1093    255
Name: merchant_title, Length: 1094, dtype: int64

In [17]:
# Since this feature consists mainly of US and China we can combine the rest into others
df['origin_country'] = df['origin_country'].replace('VE', 'Other')
df['origin_country'] = df['origin_country'].replace('AT', 'Other')
df['origin_country'] = df['origin_country'].replace('SG', 'Other')
df['origin_country'] = df['origin_country'].replace('GB', 'Other')
df['origin_country'] = df['origin_country'].replace(np.nan, 'Other')

In [18]:
df['origin_country']= LabelEncoder().fit_transform(df['origin_country']) 
df['origin_country']

0       0
1       0
2       0
3       0
4       0
       ..
1089    0
1090    0
1091    0
1092    0
1093    0
Name: origin_country, Length: 1094, dtype: int64

In [19]:
df['merchant_id']= LabelEncoder().fit_transform(df['merchant_id']) 
df['merchant_id']

0       201
1       132
2       498
3       272
4       614
       ... 
1089     24
1090    400
1091    581
1092    276
1093    509
Name: merchant_id, Length: 1094, dtype: int64

In [20]:
df['shipping_option_name']= LabelEncoder().fit_transform(df['shipping_option_name']) 
df['shipping_option_name']

0       4
1       4
2       4
3       4
4       4
       ..
1089    4
1090    4
1091    4
1092    4
1093    4
Name: shipping_option_name, Length: 1094, dtype: int64

In [21]:
df.head()

,price,retail_price,units_sold,uses_ad_boosts,rating,rating_count,badges_count,badge_local_product,badge_product_quality,badge_fast_shipping,product_color,product_variation_size_id,product_variation_inventory,shipping_option_name,shipping_option_price,shipping_is_express,countries_shipped_to,inventory_total,origin_country,merchant_title,merchant_rating_count,merchant_rating,merchant_id,merchant_has_profile_picture
0,8.00,7,10000,1,4.0,1670,0,0,0,0,86,M,50,4,3,0,30,50,0,472,66644,4.137582,201,1
1,8.00,7,1000,0,4.0,843,1,1,0,0,12,L,50,4,3,0,37,50,0,584,25752,4.256873,132,0
2,6.00,34,100,0,4.0,10,0,0,0,0,77,L,50,4,2,0,41,50,0,725,3897,3.928920,498,0
3,4.67,4,100,1,3.0,82,0,0,0,0,40,XS,5,4,1,0,25,50,0,556,7497,4.079365,272,0
4,8.00,11,1000,0,4.0,127,0,0,0,0,8,S,50,4,3,0,15,50,0,597,173,4.306358,614,0


In [22]:
# df.to_csv('/content/drive/MyDrive/colab_kagel/dataa/preprocessing.csv', encoding='utf-8', index=False)

In [23]:
df['product_variation_size_id'] = df['product_variation_size_id'].replace('S.', 'S')
df['product_variation_size_id'] = df['product_variation_size_id'].replace('XS.', 'S')
df['product_variation_size_id'] = df['product_variation_size_id'].replace('M.', 'M')
df['product_variation_size_id'] = df['product_variation_size_id'].replace('Size S', 'S')
df['product_variation_size_id'] = df['product_variation_size_id'].replace('Size-XS', 'S')
df['product_variation_size_id'] = df['product_variation_size_id'].replace('SIZE XS', 'S')
df['product_variation_size_id'] = df['product_variation_size_id'].replace('Size-S', 'S')
df['product_variation_size_id'] = df['product_variation_size_id'].replace('Size4XL', 'XL')
df['product_variation_size_id'] = df['product_variation_size_id'].replace('size S', 'S')
df['product_variation_size_id'] = df['product_variation_size_id'].replace('Size M', 'M')
df['product_variation_size_id'] = df['product_variation_size_id'].replace('Size -XXS', 'S')
df['product_variation_size_id'] = df['product_variation_size_id'].replace('SIZE-XXS', 'S')
df['product_variation_size_id'] = df['product_variation_size_id'].replace('Size S.', 'S')
df['product_variation_size_id'] = df['product_variation_size_id'].replace('s', 'S')
df['product_variation_size_id'] = df['product_variation_size_id'].replace('SizeL', 'L')
df['product_variation_size_id'] = df['product_variation_size_id'].replace('5XL', 'XL')
df['product_variation_size_id'] = df['product_variation_size_id'].replace('4XL', 'XL')
df['product_variation_size_id'] = df['product_variation_size_id'].replace('3XL', 'XL')
df['product_variation_size_id'] = df['product_variation_size_id'].replace('2XL', 'XL')

In [24]:
# list anything != name to OTHER to account for other variations
def pr_var(name):
    if name == 'XXXS' \
    or name == 'XXS' \
    or name == 'XS' \
    or name == 'S' \
    or name == 'M' \
    or name == 'L' \
    or name == 'XL' \
    or name == 'XXL' \
    or name == 'XXXXL' \
    or name == 'XXXXXL':
        return name
    else:
        return "OTHER"

In [25]:
# replace missing values and the noise in data
df['product_variation_size_id'] = df['product_variation_size_id'].replace(np.nan, 'OTHER')
# adding the new categories to df['feature']
df['product_variation_size_id'] = df['product_variation_size_id'].apply(pr_var)

In [26]:
df['product_variation_size_id']= LabelEncoder().fit_transform(df['product_variation_size_id']) 
df['product_variation_size_id']

0       1
1       0
2       0
3       5
4       3
       ..
1089    3
1090    2
1091    1
1092    1
1093    5
Name: product_variation_size_id, Length: 1094, dtype: int64

# Preprocessing on test data

In [27]:
# df2= df2.drop(df2.loc[:,df2.nunique()==1].keys(), axis=1)

In [28]:
df2= df2.drop(['currency_buyer','has_urgency_banner','theme','crawl_month','merchant_profile_picture','merchant_info_subtitle','tags','merchant_name','urgency_text','id'], axis=1)

In [29]:
df2.shape

(479, 23)

In [30]:
df2=df2.fillna(method='bfill', axis=0)

In [31]:
df2.isnull().values.any()

False

In [32]:
df2.head()

,price,retail_price,units_sold,uses_ad_boosts,rating_count,badges_count,badge_local_product,badge_product_quality,badge_fast_shipping,product_color,product_variation_size_id,product_variation_inventory,shipping_option_name,shipping_option_price,shipping_is_express,countries_shipped_to,inventory_total,origin_country,merchant_title,merchant_rating_count,merchant_rating,merchant_id,merchant_has_profile_picture
0,5.75,5,10,1,0,0,0,0,0,white,XS,5,Livraison standard,2,0,139,50,CN,liyang163,3,2.333333,5e63469b2fdc774466e15dd5,0
1,7.00,7,5000,0,579,0,0,0,0,black,XS,50,Livraison standard,2,0,31,50,CN,singing the song,4213,4.058391,583547f417dc6224cb464117,0
2,5.00,16,1000,1,62,0,0,0,0,blue,XS,20,Livraison standard,2,0,41,50,CN,Surper boutique,10501,4.029235,57086fd7fa6bee434d0e5852,0
3,18.00,27,5000,1,1903,0,0,0,0,blue,M,50,Livraison standard,5,0,42,50,CN,postonny,15855,4.052917,5a17b55d1f4d8c4f0b44b0ce,0
4,12.00,81,20000,0,1799,0,0,0,0,white,M,50,Livraison standard,3,0,38,50,CN,fashionforgirls,151914,4.127921,53aa664438d3046ee44a5024,0


In [33]:
df2['product_color']= LabelEncoder().fit_transform(df2['product_color']) 
df2['product_color']

0      50
1       5
2       8
3       8
4      50
       ..
474     5
475     8
476    50
477     5
478    55
Name: product_color, Length: 479, dtype: int64

In [34]:
df2['merchant_title']= LabelEncoder().fit_transform(df2['merchant_title']) 
df2['merchant_title']

0      250
1      321
2      114
3      286
4      187
      ... 
474     48
475     41
476    307
477    252
478    305
Name: merchant_title, Length: 479, dtype: int64

In [35]:
# Since this feature consists mainly of US and China we can combine the rest into others
df2['origin_country'] = df2['origin_country'].replace('VE', 'Other')
df2['origin_country'] = df2['origin_country'].replace('AT', 'Other')
df2['origin_country'] = df2['origin_country'].replace('SG', 'Other')
df2['origin_country'] = df2['origin_country'].replace('GB', 'Other')
df2['origin_country'] = df2['origin_country'].replace(np.nan, 'Other')

In [36]:
df2['origin_country']= LabelEncoder().fit_transform(df2['origin_country']) 
df2['origin_country']

0      0
1      0
2      0
3      0
4      0
      ..
474    1
475    0
476    1
477    0
478    0
Name: origin_country, Length: 479, dtype: int64

In [37]:
df2['merchant_id']= LabelEncoder().fit_transform(df2['merchant_id']) 
df2['merchant_id']

0      380
1      163
2       94
3      255
4        7
      ... 
474    359
475    193
476    372
477    317
478    164
Name: merchant_id, Length: 479, dtype: int64

In [38]:
df2['shipping_option_name']= LabelEncoder().fit_transform(df2['shipping_option_name']) 
df2['shipping_option_name']

0      4
1      4
2      4
3      4
4      4
      ..
474    4
475    4
476    4
477    4
478    3
Name: shipping_option_name, Length: 479, dtype: int64

In [39]:

# we will Replac the noise values in the data with the right values 

df2['product_variation_size_id'] = df2['product_variation_size_id'].replace('S.', 'S')
df2['product_variation_size_id'] = df2['product_variation_size_id'].replace('XS.', 'S')
df2['product_variation_size_id'] = df2['product_variation_size_id'].replace('M.', 'M')
df2['product_variation_size_id'] = df2['product_variation_size_id'].replace('Size S', 'S')
df2['product_variation_size_id'] = df2['product_variation_size_id'].replace('Size-XS', 'S')
df2['product_variation_size_id'] = df2['product_variation_size_id'].replace('Size-S', 'S')
df2['product_variation_size_id'] = df2['product_variation_size_id'].replace('Size4XL', 'XL')
df2['product_variation_size_id'] = df2['product_variation_size_id'].replace('size S', 'S')
df2['product_variation_size_id'] = df2['product_variation_size_id'].replace('Size M', 'M')
df2['product_variation_size_id'] = df2['product_variation_size_id'].replace('Size -XXS', 'S')
df2['product_variation_size_id'] = df2['product_variation_size_id'].replace('SIZE-XXS', 'S')
df2['product_variation_size_id'] = df2['product_variation_size_id'].replace('Size S.', 'S')
df2['product_variation_size_id'] = df2['product_variation_size_id'].replace('s', 'S')
df2['product_variation_size_id'] = df2['product_variation_size_id'].replace('SizeL', 'L')
df2['product_variation_size_id'] = df2['product_variation_size_id'].replace('5XL', 'XXXXXL')
df2['product_variation_size_id'] = df2['product_variation_size_id'].replace('4XL', 'XL')
df2['product_variation_size_id'] = df2['product_variation_size_id'].replace('3XL', 'XL')
df2['product_variation_size_id'] = df2['product_variation_size_id'].replace('2XL', 'XL')

In [40]:
# replace missing values and the noise in data
df2['product_variation_size_id'] = df2['product_variation_size_id'].replace(np.nan, 'OTHER')
# adding the new categories to df['feature']
df2['product_variation_size_id'] = df2['product_variation_size_id'].apply(pr_var)

In [41]:
df2['product_variation_size_id']= LabelEncoder().fit_transform(df2['product_variation_size_id']) 
df2['product_variation_size_id']

0      5
1      5
2      5
3      1
4      1
      ..
474    1
475    7
476    5
477    2
478    3
Name: product_variation_size_id, Length: 479, dtype: int64

In [42]:
df2.head()

,price,retail_price,units_sold,uses_ad_boosts,rating_count,badges_count,badge_local_product,badge_product_quality,badge_fast_shipping,product_color,product_variation_size_id,product_variation_inventory,shipping_option_name,shipping_option_price,shipping_is_express,countries_shipped_to,inventory_total,origin_country,merchant_title,merchant_rating_count,merchant_rating,merchant_id,merchant_has_profile_picture
0,5.75,5,10,1,0,0,0,0,0,50,5,5,4,2,0,139,50,0,250,3,2.333333,380,0
1,7.00,7,5000,0,579,0,0,0,0,5,5,50,4,2,0,31,50,0,321,4213,4.058391,163,0
2,5.00,16,1000,1,62,0,0,0,0,8,5,20,4,2,0,41,50,0,114,10501,4.029235,94,0
3,18.00,27,5000,1,1903,0,0,0,0,8,1,50,4,5,0,42,50,0,286,15855,4.052917,255,0
4,12.00,81,20000,0,1799,0,0,0,0,50,1,50,4,3,0,38,50,0,187,151914,4.127921,7,0


In [43]:
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(df2)
df2_normalized=pd.DataFrame(x_scaled, columns=df2.columns)

#functios

In [44]:
def prediction(y_data_train, x_data_test, model):    
    y_pred= model.predict(x_data_test)
    train_acc = accuracy_score(y_data_train, y_pred)
    print(train_acc)
    return y_pred

In [45]:
# Function to make precisions score, recall score, f1 score
def precision_recall_f1(y_test_pred):
    f1=f1_score(y_test, y_test_pred, average='micro')

    print('fscore',f1)

### splitting the data

In [46]:
# Split the dataset to feature coulmns(X) & label coulmn(Y)
X=df.loc[:,df.columns!='rating']
y=df.rating

In [47]:
X

,price,retail_price,units_sold,uses_ad_boosts,rating_count,badges_count,badge_local_product,badge_product_quality,badge_fast_shipping,product_color,product_variation_size_id,product_variation_inventory,shipping_option_name,shipping_option_price,shipping_is_express,countries_shipped_to,inventory_total,origin_country,merchant_title,merchant_rating_count,merchant_rating,merchant_id,merchant_has_profile_picture
0,8.00,7,10000,1,1670,0,0,0,0,86,1,50,4,3,0,30,50,0,472,66644,4.137582,201,1
1,8.00,7,1000,0,843,1,1,0,0,12,0,50,4,3,0,37,50,0,584,25752,4.256873,132,0
2,6.00,34,100,0,10,0,0,0,0,77,0,50,4,2,0,41,50,0,725,3897,3.928920,498,0
3,4.67,4,100,1,82,0,0,0,0,40,5,5,4,1,0,25,50,0,556,7497,4.079365,272,0
4,8.00,11,1000,0,127,0,0,0,0,8,3,50,4,3,0,15,50,0,597,173,4.306358,614,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1089,4.00,4,10000,0,2150,0,0,0,0,68,3,50,4,1,0,41,50,0,399,230951,4.311066,24,0
1090,5.00,5,1000,0,377,1,0,1,0,66,2,31,4,2,0,35,50,0,746,2904,4.422521,400,0
1091,9.00,48,1000,1,135,0,0,0,0,66,1,50,4,3,0,25,50,0,187,6632,3.748794,581,0
1092,11.00,10,10000,1,1479,0,0,0,0,12,1,50,4,2,0,45,50,0,708,5816,3.936382,276,0


In [48]:
y

0       4.0
1       4.0
2       4.0
3       3.0
4       4.0
       ... 
1089    4.0
1090    5.0
1091    4.0
1092    4.0
1093    3.0
Name: rating, Length: 1094, dtype: float64

In [49]:
np.unique(y)

array([1., 2., 3., 4., 5., 6.])

In [50]:
type(y)

pandas.core.series.Series

In [51]:
# y=y.astype(int)

In [52]:
y

0       4.0
1       4.0
2       4.0
3       3.0
4       4.0
       ... 
1089    4.0
1090    5.0
1091    4.0
1092    4.0
1093    3.0
Name: rating, Length: 1094, dtype: float64

In [53]:
# Splitting the dataset to training data & testing data
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.20, random_state=30, shuffle=True)

In [54]:
scaler = MinMaxScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

# SVM

In [55]:
svm = SVC(kernel='rbf', random_state =1)

In [56]:
svm.fit(x_train_scaled,y_train)

SVC(random_state=1)

In [57]:
y_pred = svm.predict(x_test_scaled)

In [58]:
f1=f1_score(y_test, y_pred, average='micro')
f1

0.7579908675799086

In [59]:
yr_pred = svm.predict(x_train_scaled)

In [60]:
f1=f1_score(y_train,yr_pred, average='micro')
f1

0.7314285714285713

In [61]:
y_test_pred = svm.predict(df2_normalized)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but SVC was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [62]:
dummy = pd.DataFrame({'id': sample['id'],'rating': y_test_pred})
dummy.to_csv('/content/drive/MyDrive/colab_kagel/dataa/svm.csv', encoding='utf-8', index=False)

# naive bayes

In [63]:
gnb = GaussianNB()
gnb.fit(x_train_scaled,y_train)
y_train_predNB=gnb.predict(x_train_scaled)

In [64]:
f1=f1_score(y_train, y_train_predNB, average='micro')
f1

0.21485714285714286

In [65]:
y_test_predNB=gnb.predict(x_test_scaled)

In [66]:
f1=f1_score(y_test, y_test_predNB, average='micro')
f1

0.182648401826484

In [67]:
y_NB = gnb.predict(df2_normalized)




/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [68]:
dummy = pd.DataFrame({'id': sample['id'],'rating': y_NB})
dummy.to_csv('/content/drive/MyDrive/colab_kagel/dataa/svm.csv', encoding='utf-8', index=False)

# naive bayes with cross validation 

In [69]:
# n_splits = 10  
# # recall is more important to us than precision in our problem
# score = f1_score

# max_recall_score = 0
# best_naive = GaussianNB()
# kf = KFold(n_splits=n_splits)


# for train_index,test_index in kf.split(x_train):
#   X_val_train, X_val_test = x_train.values[train_index], x_train.values[test_index]
#   y_val_train, y_val_test = y_train.values[train_index], y_train.values[test_index]

#   # create a new KNN model
#   naive= GaussianNB()

#   # fitting to the y_val_training data
#   naive.fit(X_val_train, y_val_train)

#   # prediction
#   y_val_pred = naive.predict(X_val_test)

#   # The score resulted from the prediction on y_val_test
#   cv_score = score(y_val_test, y_val_pred)
#   if cv_score > max_recall_score : 
#       best_naive = naive
#       max_recall_score = cv_score
        
# print("best_naive = ", best_naive,  "\nmax_recall_score = " , max_recall_score)

# Decision tree 

In [70]:
# Identifing differenet hyperparameter to tune
DT_parameter = [
    {
        'criterion': ['gini', 'entropy'],
        'max_depth': list(range(4,13,2))
    }
]

In [71]:
# Create Decision Tree classifer object
DTC = DecisionTreeClassifier()

In [72]:
# the score to DT_cv which is the best classifier is Recall
DT_cv = GridSearchCV(DTC, DT_parameter, cv=10, scoring='recall')

In [73]:
# Train Decision Tree Classifer
best_DT = DT_cv.fit(x_train_scaled,y_train).best_estimator_
best_DT

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 221, in __call__
    sample_weight=sample_weight,
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py", line 1909, in recall_score
    zero

DecisionTreeClassifier(max_depth=4)

In [74]:
print("Max_depth = ", best_DT.max_depth, "\n criterion = " , best_DT.criterion )

Max_depth =  4 
 criterion =  gini


In [75]:
#display the accuracy
print('accuracy_score on train dataset : ' )
DTC_y_train_pred = prediction(y_train, x_train_scaled, best_DT)
print('accuracy_score on test dataset : ' )
DTC_y_test_pred = prediction(y_test, x_test_scaled, best_DT)

accuracy_score on train dataset : 
0.7897142857142857
accuracy_score on test dataset : 
0.7990867579908676


In [76]:
# display precision, recall, fscore for our model
precision_recall_f1(DTC_y_test_pred)

fscore 0.7990867579908677


In [77]:
y_test_pred_DT =best_DT.predict(df2_normalized)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [78]:
dummy = pd.DataFrame({'id': sample['id'],'rating': y_test_pred_DT})
dummy.to_csv('/content/drive/MyDrive/colab_kagel/dataa/DT.csv', encoding='utf-8', index=False)